# Analyzing IMDB Data in Keras

In [1]:
# Imports
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

ModuleNotFoundError: No module named 'keras'

## 0. About
Goal: Build Neural Network Model in Keras, training and evaluating it, to analyse real data that consists of thousands of movie reviews from IMDB, and the challenge is to Predict the Sentiment Analysis of a review
based on the words in it. Use methods such as Dropout, 
Regularisation, and Optimisers to achieve a Model accuracy of 
at least 85%.

The data is a dataset of 25,000 IMDB reviews obtained from the Keras dataset list https://keras.io/datasets, each of which contains a Label where:
- Label 0 - Negative Review
- Label 1 - Positive Review

The data Input is already preprocessed where each review is 
encoded as Sequence https://keras.io/preprocessing/sequence/ of indexes corresponding to words in the review. Words are ordered
by their Frequency, where index 1 is most frequent word (i.e. "the"), index 2 is second most frequent word, and index 0 
is for unknown words.

The sentence is converted in a Vector by concatenating the integer Indexes. (i.e. given Sentence "to be or not to be" then 
the indices of the words are:
"to": 5
"be": 8
"or": 21
"not": 3

... and the Sentence is encoded as Vector [5, 8, 21, 3, 5, 8]

## 1. Loading the data
This dataset comes preloaded with Keras. The following command splits the words into training and testing data sets and labels. There is a parameter for how many words we want to look at. We've set it at 1000, but feel free to experiment.

Note that:
- `num_words` is amount of most frequent words to consider (lower value will exclude obscure words like 'ultracrepidrian')
- `skip_top` is amount of most frequence words to ignore (i.e. to ignore the word "the" which would not add value to the review, we'd set `skip_top` to a value of 2 or higher)

In [ ]:
# Loading the data (it's preloaded in Keras)
(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
    num_words=1000,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3)

print(x_train.shape)
print(x_test.shape)

## 2. Examining the data
Notice that the data has been already pre-processed, where all the words have numbers, and the reviews come in as a vector with the words that the review contains. For example, if the word 'the' is the first one in our dictionary, and a review contains the word 'the', then there is a 1 in the corresponding vector.

The output comes as a vector of 1's and 0's, where 1 is a positive sentiment for the review, and 0 is negative.

In [ ]:
print(x_train[0])
print(y_train[0])

## 3. One-hot encoding the output
Here, we'll pre-process the data by turning the input vectors into (0,1)-vectors. 

Example 1: if the pre-processed vector contains the number 14, then in the processed vector, the 14th entry will be 1.

Example 2: if we have 10 words in our vocabulary, and the vector is (4,1,8), we'll turn it into the vector (1,0,0,1,0,0,0,1,0,0)

In [ ]:
# One-hot encoding the output into vector mode, each of length 1000
tokenizer = Tokenizer(num_words=1000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print(x_train[0])

And we'll also one-hot encode the output.

In [ ]:
# One-hot encoding the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

## 4. Building the  model architecture
Build a model here using sequential. Feel free to experiment with different layers and sizes! Also, experiment adding dropout to reduce overfitting.

In [ ]:
# TODO: Build the model architecture

# TODO: Compile the model using a loss function and an optimizer.

## 5. Training the model
Run the model here. Experiment with different batch_size, and number of epochs!

In [ ]:
# TODO: Run the model. Feel free to experiment with different batch sizes and number of epochs.

## 6. Evaluating the model
This will give you the accuracy of the model, as evaluated on the testing set. Can you get something over 85%?

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: ", score[1])